In [1]:
import json
data_dir = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_clipped_processed/"+'melody.json'
queries = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_queries_processed/"+'melody.json'


In [2]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [3]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [4]:
from similarity_sankoff import similarity, Note
import miditoolkit as mtk
# queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)

In [27]:
#obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")
obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_melody.mid")


In [28]:
from Clarinet.preprocessing import getBPM

In [29]:
notes = obj.instruments[0].notes
#print(notes[:5])
end_time = notes[-1].end
print(end_time)
#tempo = getBPM(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_melody.mid")
tempo = 120
t = end_time/(4*tempo)
t = 32/768
# ans = 99999
# prev_end = 0
# for n in notes:
#     if n.start > prev_end:
#         dur = n.start - prev_end
#         ans = min(ans, dur)
#     ans = min(ans, n.end - n.start)
#     prev_end = n.end
# t= ans
        


40789


In [31]:
import numpy as np
sum= 0
start = []
end = []
for n in notes:
    start.append(n.start)
    end.append(n.end)
diff = np.array(end) - np.array(start)
print(np.mean(diff))
print(np.std(diff))

844.2894736842105
908.9625349205526


In [8]:
rep = []
prev_end = 0
for n in notes:
    if int((n.start-prev_end)/t) != 0:
        if n.start > prev_end:
            rep.append(Note("C",int((n.start-prev_end)/t),rest=True))
    if int((n.end-n.start)/t) != 0:
        rep.append(Note(midiEt_to_note[n.pitch%12 + 12],int((n.end-n.start)/t),rest=False))
    prev_end = n.end


In [49]:
#qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")
qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_query_melody.mid")


In [50]:
notes = qobj.instruments[0].notes
print(notes[:5])
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)
t=350

# ans = 99999
# prev_end = 0
# for n in notes:
#     if n.start > prev_end:
#         dur = n.start - prev_end
#         ans = min(ans, dur)
#     ans = min(ans, n.end - n.start)
#     prev_end = n.end
# t = ans


[Note(start=1792, end=3157, pitch=67, velocity=58), Note(start=3157, end=3328, pitch=60, velocity=22), Note(start=3328, end=3840, pitch=69, velocity=59), Note(start=3840, end=4096, pitch=67, velocity=63), Note(start=4096, end=5461, pitch=62, velocity=46)]


In [51]:
from tqdm import tqdm
import os
import music21
fnames = os.listdir("C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/")


def getKey(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.tonic.name)
def getMode(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.mode)


In [52]:
qrep = []
prev_end = 0
mode = getMode(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_query_melody.mid")
for n in notes:
    if int((n.start-prev_end)/t) != 0:
        if n.start > prev_end:
            qrep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
    if int((n.end-n.start)/t) != 0:
        qrep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
    prev_end = n.end

In [53]:
def compute_similarities(qrep):
    fname_to_similarity = {}
    for fname in fnames:
        sim = 0
        obj = mtk.MidiFile(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        notes = obj.instruments[0].notes 
        # ans = 99999
        # prev_end = 0
        
        # for n in notes:
        #     if n.start > prev_end:
        #         dur = n.start - prev_end
        #         ans = min(ans, dur)
        #     ans = min(ans, n.end - n.start)
        #     prev_end = n.end
        # t = ans
        t = 350
        mode = getMode(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        

        rep = []
        prev_end = 0
        for n in notes:
            if int((n.start-prev_end)/t) != 0:
                if n.start > prev_end:
                    rep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
            if int((n.end-n.start)/t) != 0:
                rep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
            prev_end = n.end
        sim = similarity(qrep, rep)
        fname_to_similarity[fname] = sim
    return fname_to_similarity


In [54]:
fname_to_similarity = compute_similarities(qrep)

21it [00:00, 647.87it/s]             
20it [00:00, 731.18it/s]             
18it [00:00, 750.13it/s]             
20it [00:00, 833.25it/s]             
26it [00:00, 1625.58it/s]            
21it [00:00, 877.59it/s]             
21it [00:00, 692.38it/s]             
20it [00:00, 793.05it/s]             
13it [00:00, 1555.93it/s]            
32it [00:00, 1229.02it/s]            
6it [00:00, 749.25it/s]              
25it [00:00, 1247.50it/s]            
20it [00:00, 1212.26it/s]            
9it [00:00, 1125.82it/s]             
27it [00:00, 1665.63it/s]            
5it [00:00, ?it/s]                   
6it [00:00, 741.85it/s]              
15it [00:00, 899.34it/s]             
23it [00:00, 880.54it/s]             
6it [00:00, 6521.33it/s]             
13it [00:00, 1490.51it/s]            
8it [00:00, 501.43it/s]              
100%|██████████| 1/1 [00:00<?, ?it/s]
32it [00:00, 982.82it/s]             
25it [00:00, 1542.59it/s]            
25it [00:00, 1041.26it/s]            
25it [00:00,

In [55]:
dict(sorted(fname_to_similarity.items(), key=lambda item: item[1], reverse=True))

{'MIDI-Unprocessed_Recital13-15_MID--AUDIO_14_R1_2018_wav--3_melody.mid': 0.9142857142857143,
 'MIDI-Unprocessed_Schubert4-6_MID--AUDIO_09_R2_2018_wav_melody.mid': 0.8857142857142857,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_19_R1_2018_wav--3_melody.mid': 0.8714285714285714,
 'MIDI-Unprocessed_Recital12_MID--AUDIO_12_R1_2018_wav--3_melody.mid': 0.8571428571428572,
 'MIDI-Unprocessed_Recital5-7_MID--AUDIO_06_R1_2018_wav--1_melody.mid': 0.8571428571428572,
 'MIDI-Unprocessed_Recital13-15_MID--AUDIO_13_R1_2018_wav--1_melody.mid': 0.8428571428571429,
 'MIDI-Unprocessed_Recital13-15_MID--AUDIO_13_R1_2018_wav--1_melody_melody.mid': 0.8428571428571429,
 'MIDI-Unprocessed_Recital13-15_MID--AUDIO_15_R1_2018_wav--4_melody.mid': 0.8428571428571429,
 'MIDI-Unprocessed_Recital5-7_MID--AUDIO_07_R1_2018_wav--4_melody.mid': 0.8428571428571429,
 'MIDI-Unprocessed_Recital8_MID--AUDIO_08_R1_2018_wav--4_melody.mid': 0.8428571428571429,
 'MIDI-Unprocessed_Schubert4-6_MID--AUDIO_10_R2_2018_wav_melody.mid'

In [26]:
similarity([Note("C", 10), Note("D", 30)], [Note("C", 10), Note("D", 50), Note("E", 30)])


100%|██████████| 2/2 [00:00<00:00, 250.01it/s]


-3.0